<a href="https://colab.research.google.com/github/sibiraj30/Mini-project-/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install nltk joblib


In [2]:
import pandas as pd
import numpy as np
import re
import joblib
import nltk
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
from google.colab import files

uploaded = files.upload()


Saving true.csv to true.csv
Saving fake.csv to fake.csv


In [6]:
# Load datasets
fake_df = pd.read_csv("fake.csv")
true_df = pd.read_csv("true.csv")

# Add labels: 0 for Fake, 1 for Real
fake_df["label"] = 0
true_df["label"] = 1

# Combine and shuffle the data
df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Preview the data
df.head()


,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [8]:
import nltk

# Download the necessary NLTK resource
nltk.download('punkt_tab')

# ... (rest of your code) ...

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Text cleaning function
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters and numbers
    text = text.lower()  # Convert to lowercase
    words = word_tokenize(text)  # Tokenize
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

# Apply cleaning
df["clean_text"] = df["text"].apply(clean_text)

# Show a few cleaned texts
df[["text", "clean_text"]].head()


,text,clean_text
0,"21st Century Wire says Ben Stein, reputable pr...",st century wire say ben stein reputable profes...
1,WASHINGTON (Reuters) - U.S. President Donald T...,washington reuters u president donald trump re...
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,reuters puerto rico governor ricardo rossello ...
3,"On Monday, Donald Trump once again embarrassed...",monday donald trump embarrassed country accide...
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",glasgow scotland reuters u presidential candid...


In [10]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])

# Convert texts to sequences
X = tokenizer.texts_to_sequences(df["clean_text"])

# Pad sequences to the same length
X = pad_sequences(X, maxlen=500, padding="post", truncating="post")

# Labels
y = df["label"].values


In [11]:
from sklearn.model_selection import train_test_split

# Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Build LSTM model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=500),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Show model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=32
)


Epoch 1/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 729s 645ms/step - accuracy: 0.5204 - loss: 0.6952 - val_accuracy: 0.5245 - val_loss: 0.6928
Epoch 2/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 740s 643ms/step - accuracy: 0.5094 - loss: 0.6936 - val_accuracy: 0.5245 - val_loss: 0.6920
Epoch 3/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 748s 647ms/step - accuracy: 0.5162 - loss: 0.6927 - val_accuracy: 0.5503 - val_loss: 0.6775
Epoch 4/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 742s 648ms/step - accuracy: 0.6440 - loss: 0.5479 - val_accuracy: 0.9905 - val_loss: 0.0445
Epoch 5/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 743s 648ms/step - accuracy: 0.9878 - loss: 0.0463 - val_accuracy: 0.9969 - val_loss: 0.0175


In [14]:
# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")

# Save the model and tokenizer
model.save("fake_news_lstm_model.h5")

import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

281/281 ━━━━━━━━━━━━━━━━━━━━ 39s 139ms/step - accuracy: 0.9978 - loss: 0.0149


Model Accuracy: 0.9969


['tokenizer.pkl']